# Computational Practicum: Resource Requirements for Gene Expression in Cell-Free Systems
Fill me in

In [1]:
include("Include.jl")

exchangereactions (generic function with 1 method)

## Task 1: Build the System Matricies
In this task, we will build the system matrices for the cell-free system. The system matrices are used to describe the interactions between the different components of the system. 

In [2]:
genesequence, proteinsequence = let

    # setup the paths to the gene and protein sequence files
    path_to_gene_file = joinpath(_PATH_TO_DATA, "deGFP.gene")
    path_to_protein_file = joinpath(_PATH_TO_DATA, "deGFP.prot")

    # load the sequences from the files - 
    gene_sequence = load_gene_sequence_from_file(path_to_gene_file); # load the gene sequence
    protein_sequence = load_protein_sequence_from_file(path_to_protein_file); # load the protein sequence

    # return -
    gene_sequence, protein_sequence
end;

Fill me in


In [6]:
reactions = let

    # initialize -
    reactions = Vector{String}();
    
    # TXTL and "hypothetical" exchange reactions -
    transcription_reactions = transcription(genesequence); # transcription reactions
    translation_reactions = translation(proteinsequence); # translation reactions
    txtl_reactions = [transcription_reactions; translation_reactions]; # combine transcription and translation reactions
    
    # build the exchange reactions -
    exchange_reactions = exchangereactions(txtl_reactions); # exchange reactions
    
    # build the list of reactions -
    reactions = [txtl_reactions; exchange_reactions]; # combine all reactions
end;

Fill me in

In [7]:
let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file

    # write the reactions to the file -
    open(path_to_reaction_file, "w") do io
        for reaction in reactions
            println(io, reaction); # write the reaction to the file
        end
    end

end

Fill me in.

In [ ]:
S, species, reactions, reaction_dictionary, bounds = let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file
    reactions = read_reaction_file(path_to_reaction_file); # read the reactions from the file
    
    # Compute the stoichiometric matrix -
    (S, species_array, reaction_array, reaction_dictionary) = build_stoichiometric_matrix(reactions); # compute the stoichiometric matrix
   
    # compute the bounds -
    bounds = build_default_bounds_array(reactions); # this is the default bounds array

    # return -
    S, species_array, reactions, reaction_dictionary, bounds
end;

## Task 2: Update the Bounds Array
Fill me in

## Task 3: Estimate the Protein Production Rate
In this task, we will estimate the protein production rate of the cell-free system and the protein concentration as a function of time and inducer concentration. 